## 7.3 浮点变量和操作符

x86系列中的现代微处理器具有两种不同类型的浮点寄存器，以及相应的两种不同类型的浮点指令。每种类型都有其优点和缺点。

执行浮点操作的原先的方法涉及到8个浮点寄存器，组织为寄存器堆栈。
这些寄存器具有长双精度（80位）。使用寄存器堆栈的优点是：

- 所有计算均以长双精度（long double）完成。
- 不同精度之间的转换不需要额外的时间。
- 对于数学函数，如对数和三角函数，内部指令来处理。
- 代码紧凑，在代码缓存中占用很少的空间。

寄存器堆栈也有缺点：
- 由于寄存器栈的组织方式，编译器难以创建寄存器变量。
- 除非启用了Pentium-II或更高版本的指令集，否则浮点比较很慢。
- 整数和浮点数之间的转换效率低下。
- 当使用长双精度时，除法，平方根等数学函数消耗更多时间用于计算。

有一种新的浮点运算方法，涉及八个或十六个向量寄存器（XMM或YMM）。这些向量寄存器可用于多种用途。
浮点运算以单精度或双精度完成，中间结果始终以与操作数相同的精度进行计算。
使用矢量寄存器的优点是：

- 创建浮点寄存器变量很容易。
- 向量操作可用来对XMM寄存器中的两个双精度变量，或四个单精度变量的向量，执行并行计算（参见第108页）。
如果AVX指令集可用，则每个矢量可以在YMM寄存器中保存四个双精度或八个单精度变量。

缺点是：
- 不支持长双精度。
- 对于操作数具有混合精度的表达式的计算，需要精确的转换指令，这可能非常耗时（请参见第144页）。
- 数学函数必须使用函数库，但这通常比内在硬件函数更快。

关于浮点寄存器堆栈和XMM，YMM的可用性：
- 所有具有浮点功能的系统，都可以使用浮点堆栈寄存器（64位Windows的设备驱动程序除外）。
- 如果系统支持SSE2或更高版本指令集（单精度仅需要SSE），XMM矢量寄存器可用于64位系统和32位系统。
- 如果处理器和操作系统支持AVX指令集，则YMM寄存器可用。 

有关如何测试这些指令集的可用性，请参阅第125页。

**大多数编译器在浮点计算都会使用XMM寄存器进行浮点运算，只要条件支持，即64位模式或SSE2指令集启用时。**
少部分编译器能够支持这两种类型的浮点运算的混合，并为每种计算选择最优的类型。

在大多数情况下，**双精度计算不会比单精度花费更多的时间。**
当使用浮点寄存器时，单精度和双精度之间的速度没有区别。
长双精度（long double）只需要稍微多一点的时间。
在使用XMM寄存器时，对于大多数处理器，当不用矢量运算时，
- 单精度除法，平方根和数学函数的计算速度快于双精度，
- 而加，减，乘等速度仍然相同，无论精度如何。

所以：
- **如果对应用程序有好处，您可以使用双精度而不必担心成本太高。**
- **如果您有大数组并希望尽可能多地将数据存入数据缓存，则可以使用单精度。**
- **如果您可以利用矢量操作，单精度是很好的选择，如第108页所述。**


取决于微处理器的不同，
- 浮点加法需要3至6个时钟周期。
- 乘法需要4至8个时钟周期。
- 除法需要14-45个时钟周期。

使用浮点堆栈寄存器时，**浮点比较指令效率低下。**
使用浮点堆栈寄存器时，**浮点或双精度浮点转换为整数需要很长时间。**
在使用XMM寄存器时，**不要混合使用单精度和双精度。**见第144页。

如果可能，**避免整数和浮点变量之间的转换**。见第144页。

在XMM寄存器中产生浮点下溢的应用程序，可以设置`flush-to-zero`模式（清洗至零模式），而不是在下溢的情况下生成非规格化（subnormal）小数：

```cpp
// Example 7.5. Set flush-to-zero mode (SSE):
#include <xmmintrin.h>
_MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
```

强烈建议设置`flash-to-zero`模式，除非你有特殊原因使用非规格化小数。
另外，如果SSE2可用，你可以设置`denormals-are-zero`模式：

It is strongly recommended to set the flush-to-zero mode unless you have special reasons
to use subnormal numbers.

You may, in addition, set the denormals-are-zero mode if SSE2 is available:

```cpp
// Example 7.6. Set flush-to-zero and denormals-are-zero mode (SSE2):
#include <xmmintrin.h>
_mm_setcsr(_mm_getcsr() | 0x8040);
```

有关数学函数的更多信息，请参阅第149和122页。

译者注（来自网络）：
- 在SSE和SSE2指令集中，有两种模式FTZ( Flush-to-Zero)和DAZ(Denormals-Are-Zero)帮助我们提高非规格化小数的运算速度。
- 其中FTZ的意思是当运算结果产生非规格化小数时，FTZ模式将运算结果设置为0。
- 而DAZ的意思是当操作数有非规格化小数时，先将它设置为0再与其他操作数进行运算。
- 也可以说FTZ影响输出结果而DAZ影响输入结果。